# Google Cloud Parallel Data Read Speeds with Dask

## Imports & Client Initialization

In [1]:
import dask.array as dsa
import fsspec
import numpy as np
import dask.dataframe as dd
from contextlib import contextmanager
#import intake
import time
#import seaborn as sns
import dask
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\lib\site-packages\distributed\protocol\core.py", line 107, in loads
    return msgpack.loads(
  File "C:\Users\HP\anaconda3\lib\site-packages\msgpack\fallback.py", line 118, in unpackb
    unpacker = Unpacker(None, **kwargs)
TypeError: __init__() got an unexpected keyword argument 'strict_map_key'
distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\lib\site-packages\distributed\protocol\core.py", line 107, in loads
    return msgpack.loads(
  File "C:\Users\HP\anaconda3\lib\site-packages\msgpack\fallback.py", line 118, in unpackb
    unpacker = Unpacker(None, **kwargs)
TypeError: __init__() got an unexpected keyword argument 'strict_map_key'


RuntimeError: Cluster failed to start. Timed out during handshake while connecting to tcp://127.0.0.1:61675 after 10 s

## Benchmarking Setup

This class is taken directly from GitHub (ec_20_aberanthey_etal / cloud_storage.ipynb)

The Diagnostic Timer will keep track of data retrieval times and store them within a pandas dataframe for processing.

In [ ]:
class DiagnosticTimer:
    def __init__(self):
        self.diagnostics = []
        
    @contextmanager
    def time(self, **kwargs):
        tic = time.time()
        yield
        toc = time.time()
        kwargs["runtime"] = toc - tic
        self.diagnostics.append(kwargs)
        
    def dataframe(self):
        return pd.Dataframe(self.diagnostics)
    
diag_timer = DiagnosticTimer()

## Perform Benchmarking

In [ ]:
token = '/pw/notebook-lab/used-cars-key.json'

### CSV

#### DataFrame

Note that CSV files **cannot** be directly read into a Dask array. If you want the CSV file in a Dask array, there are 2 options:

* Load CSV into numpy array, then convert numpy array to Dask array
* Load CSV into Dask Dataframe, then convert the dataframe into an array

This will be much slower than simply staying with the dataframe option, so if you want a Dask array make sure the file is stored in Zarr format in Google Cloud Storage.

In [ ]:
def total_nthreads():
    return sum([v for v in client.nthreads().values()])

def total_ncores():
    return sum([v for v in client.ncores().values()])

def total_workers():
    return len(client.ncores())

This is the main loop, pulled from (ec_20_abernathy_etal/cloud_storage.ipynb)

In [ ]:
diag_kwargs = dict(nbytes=Path('/pw/storage/step_09_output.csv').stat().st_size, format='csv')

for nworkers in [5, 10, 20, 30]:
    cluster.scale(nworkers)
    time.sleep(10)
    client.wait_for_workers(nworkers)
    print(nworkers)
    with diag_timer.time(nthreads=total_nthreads(), ncores=total_ncores(), nworkers=total_workers(), **diag_kwargs):
        df = dd.read_csv('gs://used-cars-test/step_09_output.csv', storage_options={'token':token})

In [ ]:
client.close()
cluster.close()